# Main notebook for battery state estimation

In [1]:
import numpy as np
import pandas as pd
import scipy.io
import math
import os
import ntpath
import sys
import logging
import time
import sys

from importlib import reload
import plotly.graph_objects as go

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam
from keras.utils import np_utils
from keras.layers import LSTM, Embedding, RepeatVector, TimeDistributed, Masking
from keras.callbacks import EarlyStopping, ModelCheckpoint, LambdaCallback


IS_COLAB = False

if IS_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    data_path = "/content/drive/My Drive/battery-state-estimation/battery-state-estimation/"
else:
    data_path = "../../"

sys.path.append(data_path)
from data_processing.lg_dataset import LgData

Using TensorFlow backend.


### Config logging

In [2]:
reload(logging)
logging.basicConfig(format='%(asctime)s [%(levelname)s]: %(message)s', level=logging.DEBUG, datefmt='%Y/%m/%d %H:%M:%S')

# Load Data

In [3]:
train_names = [
    '25degC/551_LA92', 
    '25degC/551_Mixed1', 
    '25degC/551_Mixed2', 
    '25degC/551_UDDS', 
    '25degC/551_US06', 
    '25degC/552_Mixed3',

    '25degC/552_Mixed7', 
    '25degC/552_Mixed8', 
    ]
test_names = [
    '25degC/552_Mixed4', 
    '25degC/552_Mixed5', 
    '25degC/552_Mixed6', 
    ]

steps = 300

lg_data = LgData(data_path)
cycles = lg_data.get_discharge_whole_cycle(train_names, test_names, output_capacity=False)
train_x, train_y, test_x, test_y = lg_data.get_discharge_multiple_step(cycles, steps)

train_y = lg_data.keep_only_y_end(train_y, steps)
test_y = lg_data.keep_only_y_end(test_y, steps)

2020/12/07 16:21:35 [INFO]: Train x: (2136, 300, 3), train y: (2136, 300, 1) | Test x: (709, 300, 3), test y: (709, 300, 1)


In [4]:
# Model definition

#opt = tf.keras.optimizers.Adam(lr=0.00001)

#model = Sequential()
#model.add(LSTM(256, activation='selu',
#                return_sequences=True,
#                input_shape=(train_x.shape[1], train_x.shape[2])))
#model.add(LSTM(256, activation='selu', return_sequences=False))
#model.add(Dense(256, activation='selu'))
#model.add(Dense(128, activation='selu'))
#model.add(Dense(1, activation='linear'))
#model.summary()

#model.compile(optimizer=opt, loss='huber', metrics=['mse', 'mae', 'mape', tf.keras.metrics.RootMeanSquaredError(name='rmse')

In [5]:
experiment_name = '2020-12-06-19-44-14_lstm_soc_lg'

history = pd.read_csv(data_path + 'results/trained_model/%s_history.csv' % experiment_name)
model = keras.models.load_model(data_path + 'results/trained_model/%s.h5' % experiment_name)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 300, 256)          266240    
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense (Dense)                (None, 256)               65792     
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 890,369
Trainable params: 890,369
Non-trainable params: 0
_________________________________________________________________


In [6]:
print(history)

    Unnamed: 0      loss       mse       mae       mape      rmse  val_loss  \
0            0  0.007985  0.015969  0.086286  52.072884  0.126369  0.000560   
1            1  0.000647  0.001295  0.028370  12.785918  0.035986  0.000469   
2            2  0.000446  0.000893  0.022999  10.227242  0.029876  0.000361   
3            3  0.000401  0.000802  0.021868   9.565425  0.028311  0.000369   
4            4  0.000347  0.000694  0.020349   8.510178  0.026346  0.000330   
..         ...       ...       ...       ...        ...       ...       ...   
95          95  0.000154  0.000308  0.014032   7.002544  0.017540  0.000262   
96          96  0.000151  0.000302  0.013452   6.005038  0.017375  0.000264   
97          97  0.000122  0.000243  0.012013   5.263407  0.015600  0.000142   
98          98  0.000097  0.000193  0.010652   4.854163  0.013894  0.000136   
99          99  0.000131  0.000263  0.012581   5.561449  0.016216  0.000188   

     val_mse   val_mae   val_mape  val_rmse  
0   0

### Testing

In [7]:
results = model.evaluate(test_x, test_y)
print(results)

23/23 [==============================] - 7s 319ms/step - loss: 2.1433e-04 - mean_squared_error: 4.2867e-04 - mean_absolute_error: 0.0156 - mean_absolute_percentage_error: 7.9237 - rmse: 0.0207
[0.00021433422807604074, 0.0004286684561520815, 0.015640864148736, 7.92370080947876, 0.02070431038737297]


# Data Visualization

In [8]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=history['loss'],
                    mode='lines', name='train'))
fig.add_trace(go.Scatter(y=history['val_loss'],
                    mode='lines', name='validation'))
fig.update_layout(title='Loss trend',
                  xaxis_title='epoch',
                  yaxis_title='loss',
                  width=1400,
                  height=600)
fig.show()

In [9]:
train_predictions = model.predict(train_x)

In [10]:
cycle_num = 0
steps_num = 8000
step_index = np.arange(cycle_num*steps_num, (cycle_num+1)*steps_num)

fig = go.Figure()
fig.add_trace(go.Scatter(x=step_index, y=train_predictions.flatten()[cycle_num*steps_num:(cycle_num+1)*steps_num],
                    mode='lines', name='SoC predicted'))
fig.add_trace(go.Scatter(x=step_index, y=train_y.flatten()[cycle_num*steps_num:(cycle_num+1)*steps_num],
                    mode='lines', name='SoC actual'))
fig.update_layout(title='Results on training',
                  xaxis_title='Step',
                  yaxis_title='SoC percentage',
                  width=1400,
                  height=600)
fig.show()

In [11]:
test_predictions = model.predict(test_x)

In [12]:
cycle_num = 0
steps_num = 8000
step_index = np.arange(cycle_num*steps_num, (cycle_num+1)*steps_num)

fig = go.Figure()
fig.add_trace(go.Scatter(x=step_index, y=test_predictions.flatten()[cycle_num*steps_num:(cycle_num+1)*steps_num],
                    mode='lines', name='SoC predicted'))
fig.add_trace(go.Scatter(x=step_index, y=test_y.flatten()[cycle_num*steps_num:(cycle_num+1)*steps_num],
                    mode='lines', name='SoC actual'))
fig.update_layout(title='Results on testing',
                  xaxis_title='Step',
                  yaxis_title='SoC percentage',
                  width=1400,
                  height=600)
fig.show()